#### Q1. Getting the embeddings model

In [7]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [8]:
user_question = "I just discovered the course. Can I still join it?"

In [9]:
user_question_emb = embedding_model.encode(user_question).tolist()
user_question_emb[0]

0.07822265475988388

In [10]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [11]:
documents_ml = [doc for doc in documents if doc['course'] == "machine-learning-zoomcamp"]

In [12]:
len(documents_ml)

375

#### Q2. Creating the embeddings

In [15]:
import numpy as np

In [13]:
embeddings = []
for doc in documents_ml:
    qa_text = f"{doc['question']} {doc['text']}"
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

In [16]:
X = np.array(embeddings)

In [17]:
X.shape

(375, 768)

#### Q3. Search

In [20]:
v = user_question_emb

In [21]:
scores = X.dot(v)
max(scores)

0.6506573240979582

In [29]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents_ml, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

#### Q4. Hit-rate for our search engine

In [1]:
from tqdm.auto import tqdm
import pandas as pd

In [2]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [24]:
def question_vector(question):
    v_q = embedding_model.encode(question)
    return search_engine.search(v_q, num_results=5)

In [30]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = question_vector(q['question'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [26]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [32]:
hit_rate(relevance_total)

0.9398907103825137

#### Q5. Indexing with Elasticsearch

In [33]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200', timeout=600) 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "qt_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

/tmp/ipykernel_7724/690416575.py:3: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es_client = Elasticsearch('http://localhost:9200', timeout=600)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': False, 'index': 'course-questions'})

In [ ]:
for doc in tqdm(documents_ml):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/375 [00:00<?, ?it/s]

ApiError(503, 'unavailable_shards_exception', '[course-questions][0] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[course-questions][0]] containing [index {[course-questions][YGmeppAB3mMnWrULXRTE], source[{"text":"Machine Learning Zoomcamp FAQ\\nThe purpose of this document is to capture frequently asked technical questions.\\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\\nData Engineering Zoomcamp FAQ\\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\\nwork","section":"General course-related questions","question":"How do I sign up?","course":"machine-learning-zoomcamp","id":"0227b872"}]}]]')


In [ ]:
def elastic_search_knn(field, vector):
    
    knn_query = {
    "field": "qt_vector",
    "query_vector": vector,
    "k": 5,
    "num_candidates": 10000
    }

    search_query = {
        "knn": knn_query,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
def question_text_vector_knn(question):
    v_q = embedding_model.encode(question)
    return elastic_search_knn('question_text_vector', v_q)

In [ ]:
question_text_vector_knn(user_question)

Highest score id: ee58a693

#### Q6. Hit-rate for Elasticsearch

In [ ]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = question_text_vector_knn(q['question'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [ ]:
hit_rate(relevance_total)

0.9398907103825137